# Preprocessing a dataset

## Setup
### Imports

In [ ]:
import pandas as pd                                     # for dataset manipulation (DataFrames)
import numpy as np                                      # allows some mathematical operations
import matplotlib.pyplot as plt                         # library used to display graphs
import seaborn as sns                                   # more convenient visualisation library for dataframes
import missingno as msno                                # utility library for missing values
from sklearn.model_selection import train_test_split    # for classification
from sklearn.neighbors import KNeighborsClassifier      # for classification

### Loading the dataset

In [ ]:
df = pd.read_csv("diabetes_data.csv")
data_columns = list(df.columns)

## Data preparation
### Preliminary analysis

Using what we learned last time, familiarize yourself with the diabetes dataset.
What observations can you make ? What problem do you think we are trying to solve with this data ?

In [ ]:
# Your code here

*[Your answers here]*

### Objectives and baseline method
For the rest of this practical, we will use a K-nearest-neighbours (KNN) algorithm to make a diagnosis from the dataset. This means that today, the only way we can improve this diagnosis is by pre-processing the data.
The accuracy of the algorithm will be our metric to evaluate the quality of our pre-processing.

Below is the function you will need to use the algorithm. You don't need to understand it yet - it will all be explained in future lessons !

In [ ]:
def split_data(data):
    X = data.drop("Outcome", axis=1)
    y = data.Outcome
    return train_test_split(X, y,
                            test_size=0.33,  # 10% of the data will be used for testing
                            random_state=42,  # ensures reproducibility of the test
                            stratify=y  # ensures the proportion of ill people is the same
                            )

def print_knn_score(scores, data_type=""):
    max_score = max(scores)
    k_values_max_score = [i + 1 for i, v in enumerate(scores) if v == max_score]
    print(f'Max {data_type} score {max_score * 100} % for k = {[i for i in k_values_max_score]}')

def diagnosis_knn(data):
    """ KNN-based classification for diabetes diagnosis. """
    X_train, X_test, y_train, y_test = split_data(data)
    test_scores = []
    train_scores = []

    for k in range(1, 15):
        knn = KNeighborsClassifier(k)
        knn.fit(X_train, y_train)
        train_scores.append(knn.score(X_train, y_train))  # "score" for KNN is the accuracy of the classification
        test_scores.append(knn.score(X_test, y_test))

    print_knn_score(train_scores, "train")
    print_knn_score(test_scores, "test")

With this, we can have a "baseline" score, *i.e.* a score we can refer to in the future to see how performance evolves with our changes.

In [ ]:
diagnosis_knn(df)

### Scaling the data

KNN is a distance-based algorithm, meaning it is can sensitive to the absolute values of the features. One possible way to counter this issue is to standardize the data (subtract mean and divide by standard deviation).

Be careful not to mix up **normalization** and **standardization**!
**Normalizing data** means dividing each value by its norm. There are several norms (l1, l2), which all have different properties.

**Normalization changes the distribution of the data**, whereas **standardization simply changes its scale**.

#### Task
Apply each of the following operations. For each of them, check how the prediction score changes, and try to interpret it.
- Standardize the data.
- Normalize the data. Make sure to use the original data, and not the standardized data!
- In a separate variable, use both standardization and normalization.
- *Bonus* : You can check out the [`scikit-learn` API](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing) and use one of the other scalers listed there, such as `MinMaxScaler` or `MaxAbsScaler`. Think of use cases for these scalers.

*Hints*:
- You can either try to define these functions yourself, or you can use `scikit-learn`'s dedicated objects : [`StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) and [`Normalizer`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Normalizer.html#sklearn.preprocessing.Normalizer). Refer to the documentation for information on how to use them. Don't forget the imports!
- It is advised to create a function that will do the scaling for you. This way, you can easily scale other datasets in the rest of the practical.

In [ ]:
# Example for StandardScaler ----------------------------------------------------------------------
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()                                   # create an instance of the scaler
df_std = df.drop(["Outcome"], axis=1, inplace=False)        # create a copy excluding target
df_std = scaler.fit_transform(df_std)                       # fit the scaler
df_std = pd.DataFrame(df_std, columns=data_columns[:-1])    # transform it to dataframe
df_std["Outcome"] = df["Outcome"]                           # add the outcome column back
# -------------------------------------------------------------------------------------------------

In [ ]:
# Your code here

*[Your comments here]*


In [ ]:
def scale(data):
    scaled_data = ...
    ...                        # perform some pre-processing here
    return scaled_data

### Managing missing values

#### "What missing values?"

Using the functions we saw in the last practical, determine (if you have not already) whether the dataset contains missing values.
Now observe the minimum and maximum values for each feature. Does anything seem weird to you? Try to interpret the abnormalities.

In [ ]:
# Your code here

*[Your answers here]*

#### Cleaning the data

You must now have noticed that there are indeed missing values in the dataset. To avoid creating bias in the data and misleading both analysis and algorithms, it is actually better to replace those values by actual missing values, which we will manage later on.

In the code cell below, replace the missing values with `NaN` types, meaning "Not a Number".
Make sure to use the original data, not the scaled data from the previous section!
It is a good idea to scale data, but the scaling needs to be done after the cleaning. In order to compare the results with and without scaled data, you will need to re-scale the data everytime we impute a different value for `NaN`s.

*Hints*:
- Make sure to only replace values in the relevant columns!
- You can use the `replace()` function to perform the replacement
- You can indicate a `NaN` type with `np.NaN`

In [ ]:
# This syntax lets you select several columns at once
# df[['column1','column2','column3']]

# Your code here

#### Observe missing data

The `missingno` library lets you visualize where `NaN`s are in your dataset. Sometimes, it lets you determine whether your data is missing randomly or not.

In [ ]:
msno.matrix(df)

### Deleting rows with missing values

As a first approach, try deleting rows where data is missing. How much data do you have left?
Check the new score. How do you interpret this result?

In [ ]:
# Your code here

*[Your comments here]*

### Replacing missing values

In a lot of cases, replacing the missing data can provide better classification results than simply deleting the data. In the sections below, we will test different methods for data replacement.

For each of these methods :
- Perform the replacement on the data we cleaned up
- Check how the diagnosis score evolves
- Try providing an explanation for this evolution
- Scale the data and see if there are any changes

*Hint: Pandas has a `fillna()` function!*

#### Replacing with mean values

In [ ]:
# Your code here

*[Your comments here]*

#### Replacing with median values

In [ ]:
# Your code here

*[Your comments here]*

#### Replacing with random values

Here are some ways you can replace with random values:
- Replace with completely random values
- Replace with random values drawn uniformly between the minimum and maximum of each column
- Replace with random values drawn from a certain distribution for each column

*Hints*:
  - Observing the data with a `displot` (with `kind="kde"`) can help you guess what type of distribution you need
  - You can use functions such as [`numpy.random.uniform`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.uniform.html) to draw random samples. Numpy has functions for many distributions, such as uniform, normal or exponential.

In [ ]:
# Your code here

*[Your comments here]*

#### Scikit learn imputation

For this step, you can try using `scikit-learn`'s `SimpleImputer`, `KNN-Imputer` and / or `IterativeImputer`. Make sure you refer to the documentation to parametrize the imputers!

In [ ]:
# Your code here

*[Your comments here]*

#### Mixing and matching

You are not obligated to use the same pre-processing for every column. In the cell below, try combining different methods in order to get the best classification possible.

In [ ]:
# Your code here

*[Your comments here]*